In [1]:
#Import external functions
from image_processing import *
from video_processing import *
from utils import *
from custom_models import *
import pickle
import random
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint

Using TensorFlow backend.


In [2]:
# Load images for training
data_type = 'flow_images'
weight_file_path = 'saved_models/caffenet_single_flow.hdf5'

classes_dictionary = pickle.load(open('data_a/action_dictionary.p','rb'))

In [3]:
# Single frame model definition
input_shape = (crop_height,crop_width,3)
seq_input = (seq_len,crop_height,crop_width,3)

#SGD Optimizer Parameters
base_lr = 0.001
momentum = 0.9

#Step-based LearningRate Parameters
epochs = 420
gamma = 0.1
stepsize = 210

In [4]:
#Create CNN-LSTM model
lstm_model = LSTMCaffeDonahueFunctional(seq_input=seq_input, num_labels=num_labels,check1024=True)

#Load Single Frame weights for faster training
lstm_model = load_single_frame_weights(lstm_model, weight_file_path)

# Configure the model for training
lstm_model = model_compile(model=lstm_model, base_lr=base_lr, momentum=momentum)

In [5]:
file = open('data_a/ucf101_split1_trainVideos.txt','r')
train_videos = file.readlines()
file.close()
N = len(train_videos)
random.shuffle(train_videos)
val_videos = train_videos[:N//3]
train_videos = train_videos[N//3:]

batch_size_train = 17
batch_size_val = 17

In [6]:
#### STEP-BASED DECAY
# To Keras ston SGD apo mono tou kanei time-based decay
# Protimoume Step decay, giati emfanizei kalutero convergence
# Isws 8a mporousa na dokimasw kai Adaptive Learning
#
# Possible solution : https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
# Possible solution : https://machinelearningmastery.com/using-learning-rate-schedules-deep-learning-models-python-keras/
####
def step_decay(epoch):
    initial_lrate = base_lr
    drop = gamma
    epochs_drop = stepsize
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))

In [7]:
for pseudoseed in range(iterations):
    train_generator = frame_generator(train_videos,batch_size_train,data_type,pseudoseed)
    val_generator = frame_generator(val_videos,batch_size_val,data_type,pseudoseed)

    filepath = 'saved_models/lstm_flow_1024_pseudo'+str(pseudoseed)+'.hdf5'
    checkpointer = ModelCheckpoint(filepath=filepath,verbose=1,save_best_only=True)
    loss_history = LossHistory()
    lrate = LearningRateScheduler(step_decay,verbose=1)

    history = lstm_model.fit_generator(generator=train_generator,
                                       steps_per_epoch=15,
                                       validation_data=val_generator,
                                       validation_steps=5,
                                       epochs=epochs,
                                       callbacks=[lrate,loss_history,checkpointer])

Epoch 1/420

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 41s 3s/step - loss: 5.3400 - acc: 0.0196 - top_k_categorical_accuracy: 0.0706 - val_loss: 5.3295 - val_acc: 0.0353 - val_top_k_categorical_accuracy: 0.1294

Epoch 00001: val_loss improved from inf to 5.32947, saving model to saved_models/lstm_flow_1024_pseudo0.hdf5
Epoch 2/420

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 5.3162 - acc: 0.0275 - top_k_categorical_accuracy: 0.1451 - val_loss: 5.2818 - val_acc: 0.0824 - val_top_k_categorical_accuracy: 0.2588

Epoch 00002: val_loss improved from 5.32947 to 5.28177, saving model to saved_models/lstm_flow_1024_pseudo0.hdf5
Epoch 3/420

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 5.2783 - acc: 0.0902 - top_k_categorical_accuracy: 0.2549 - val_loss: 5.2560 - va

Epoch 23/420

Epoch 00023: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 3.3365 - acc: 0.3922 - top_k_categorical_accuracy: 0.6784 - val_loss: 3.2595 - val_acc: 0.4000 - val_top_k_categorical_accuracy: 0.6471

Epoch 00023: val_loss improved from 3.40679 to 3.25949, saving model to saved_models/lstm_flow_1024_pseudo0.hdf5
Epoch 24/420

Epoch 00024: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 3.3537 - acc: 0.3490 - top_k_categorical_accuracy: 0.6627 - val_loss: 2.8682 - val_acc: 0.4824 - val_top_k_categorical_accuracy: 0.7294

Epoch 00024: val_loss improved from 3.25949 to 2.86824, saving model to saved_models/lstm_flow_1024_pseudo0.hdf5
Epoch 25/420

Epoch 00025: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 3.1948 - acc: 0.3765 - top_k_categorical_accuracy: 0.7294 - val_loss: 2.79

15/15 [==============================] - 28s 2s/step - loss: 2.4851 - acc: 0.5373 - top_k_categorical_accuracy: 0.8549 - val_loss: 2.4024 - val_acc: 0.5176 - val_top_k_categorical_accuracy: 0.8235

Epoch 00046: val_loss did not improve from 2.13417
Epoch 47/420

Epoch 00047: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 2.2891 - acc: 0.6078 - top_k_categorical_accuracy: 0.8745 - val_loss: 2.4601 - val_acc: 0.5765 - val_top_k_categorical_accuracy: 0.8235

Epoch 00047: val_loss did not improve from 2.13417
Epoch 48/420

Epoch 00048: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 2.4209 - acc: 0.6000 - top_k_categorical_accuracy: 0.8510 - val_loss: 2.6429 - val_acc: 0.5412 - val_top_k_categorical_accuracy: 0.8235

Epoch 00048: val_loss did not improve from 2.13417
Epoch 49/420

Epoch 00049: LearningRateScheduler setting learning rate to 0.001.
15/15 [===


Epoch 00070: val_loss did not improve from 2.02153
Epoch 71/420

Epoch 00071: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 2.1312 - acc: 0.6275 - top_k_categorical_accuracy: 0.8980 - val_loss: 1.9939 - val_acc: 0.7176 - val_top_k_categorical_accuracy: 0.9059

Epoch 00071: val_loss improved from 2.02153 to 1.99391, saving model to saved_models/lstm_flow_1024_pseudo0.hdf5
Epoch 72/420

Epoch 00072: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 2.1235 - acc: 0.6667 - top_k_categorical_accuracy: 0.8824 - val_loss: 2.1646 - val_acc: 0.6588 - val_top_k_categorical_accuracy: 0.8941

Epoch 00072: val_loss did not improve from 1.99391
Epoch 73/420

Epoch 00073: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 2.0126 - acc: 0.6392 - top_k_categorical_accuracy: 0.9137 - val_loss: 1.8841 - val_a

15/15 [==============================] - 28s 2s/step - loss: 1.9768 - acc: 0.6824 - top_k_categorical_accuracy: 0.9333 - val_loss: 2.2394 - val_acc: 0.5294 - val_top_k_categorical_accuracy: 0.9059

Epoch 00095: val_loss did not improve from 1.79042
Epoch 96/420

Epoch 00096: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 1.9743 - acc: 0.6471 - top_k_categorical_accuracy: 0.9098 - val_loss: 1.7424 - val_acc: 0.7176 - val_top_k_categorical_accuracy: 0.9412

Epoch 00096: val_loss improved from 1.79042 to 1.74242, saving model to saved_models/lstm_flow_1024_pseudo0.hdf5
Epoch 97/420

Epoch 00097: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 28s 2s/step - loss: 1.9570 - acc: 0.6902 - top_k_categorical_accuracy: 0.9176 - val_loss: 2.3987 - val_acc: 0.6471 - val_top_k_categorical_accuracy: 0.8471

Epoch 00097: val_loss did not improve from 1.74242
Epoch 98/420

Epoch 00098: Le

...

15/15 [==============================] - 28s 2s/step - loss: 0.9591 - acc: 0.9725 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.4937 - val_acc: 0.8353 - val_top_k_categorical_accuracy: 0.9765

Epoch 00309: val_loss did not improve from 1.11802
Epoch 310/420

Epoch 00310: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 28s 2s/step - loss: 0.9731 - acc: 0.9765 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.3453 - val_acc: 0.8471 - val_top_k_categorical_accuracy: 0.9647

Epoch 00310: val_loss did not improve from 1.11802
Epoch 311/420

Epoch 00311: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 28s 2s/step - loss: 0.9416 - acc: 0.9725 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.4998 - val_acc: 0.8353 - val_top_k_categorical_accuracy: 0.9529

Epoch 00311: val_loss did not improve from 1.11802
Epoch 312/420

Epoch 00312: LearningRateScheduler setting learning rate to 0.0001.
15/1

15/15 [==============================] - 28s 2s/step - loss: 0.9499 - acc: 0.9765 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.5827 - val_acc: 0.7882 - val_top_k_categorical_accuracy: 0.9647

Epoch 00334: val_loss did not improve from 1.11802
Epoch 335/420

Epoch 00335: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 28s 2s/step - loss: 0.9600 - acc: 0.9725 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.3795 - val_acc: 0.7765 - val_top_k_categorical_accuracy: 0.9765

Epoch 00335: val_loss did not improve from 1.11802
Epoch 336/420

Epoch 00336: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 28s 2s/step - loss: 0.9773 - acc: 0.9725 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.6296 - val_acc: 0.8000 - val_top_k_categorical_accuracy: 0.9412

Epoch 00336: val_loss did not improve from 1.11802
Epoch 337/420

Epoch 00337: LearningRateScheduler setting learning rate to 0.0001.
15/1

15/15 [==============================] - 28s 2s/step - loss: 0.9536 - acc: 0.9647 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.4112 - val_acc: 0.8471 - val_top_k_categorical_accuracy: 0.9765

Epoch 00359: val_loss did not improve from 1.11802
Epoch 360/420

Epoch 00360: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 28s 2s/step - loss: 0.9936 - acc: 0.9529 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.4069 - val_acc: 0.8588 - val_top_k_categorical_accuracy: 0.9647

Epoch 00360: val_loss did not improve from 1.11802
Epoch 361/420

Epoch 00361: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 28s 2s/step - loss: 0.9706 - acc: 0.9647 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.4640 - val_acc: 0.8471 - val_top_k_categorical_accuracy: 0.9647

Epoch 00361: val_loss did not improve from 1.11802
Epoch 362/420

Epoch 00362: LearningRateScheduler setting learning rate to 0.0001.
15/1

15/15 [==============================] - 28s 2s/step - loss: 0.9584 - acc: 0.9608 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.5635 - val_acc: 0.8118 - val_top_k_categorical_accuracy: 0.9647

Epoch 00383: val_loss did not improve from 1.02823
Epoch 384/420

Epoch 00384: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 28s 2s/step - loss: 0.9704 - acc: 0.9686 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.6608 - val_acc: 0.8118 - val_top_k_categorical_accuracy: 0.9647

Epoch 00384: val_loss did not improve from 1.02823
Epoch 385/420

Epoch 00385: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 28s 2s/step - loss: 0.9905 - acc: 0.9647 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.6591 - val_acc: 0.7765 - val_top_k_categorical_accuracy: 0.9529

Epoch 00385: val_loss did not improve from 1.02823
Epoch 386/420

Epoch 00386: LearningRateScheduler setting learning rate to 0.0001.
15/1

15/15 [==============================] - 28s 2s/step - loss: 0.9497 - acc: 0.9725 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.9694 - val_acc: 0.8235 - val_top_k_categorical_accuracy: 0.9529

Epoch 00408: val_loss did not improve from 1.02823
Epoch 409/420

Epoch 00409: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 28s 2s/step - loss: 0.9592 - acc: 0.9804 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.3702 - val_acc: 0.8471 - val_top_k_categorical_accuracy: 0.9647

Epoch 00409: val_loss did not improve from 1.02823
Epoch 410/420

Epoch 00410: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 28s 2s/step - loss: 0.9518 - acc: 0.9725 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.3777 - val_acc: 0.8706 - val_top_k_categorical_accuracy: 0.9765

Epoch 00410: val_loss did not improve from 1.02823
Epoch 411/420

Epoch 00411: LearningRateScheduler setting learning rate to 0.0001.
15/1